# **Я начну реализацию с импорта всех необходимых нам имён. Хоть их и можно импортировать по мере надобности в программе, однако PEP8 рекомендует делать это в начале файла.**

In [ ]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import scale
from sklearn import datasets
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from catboost import CatBoostClassifier, Pool

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# Импортируем данные

In [ ]:
data = pd.read_csv('/kaggle/input/tabular-playground-series-may-2021/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-may-2021/test.csv')
submission = pd.read_csv('/kaggle/input/tabular-playground-series-may-2021/sample_submission.csv')

# **Посмотрим на данные** 
## Отсюда стартует блокнот

In [ ]:
data.head(10)

In [ ]:
test.head()

### id'шник нам не нужен, он здесь в роли индекса, поэтому просто удалим его из данных

In [ ]:
data = data.drop('id', axis=1)
test = test.drop('id', axis=1)

In [ ]:
data.shape, test.shape

In [ ]:
data.info()

Пропусков в данных нет. Хотя по-моему мнение, можно было добавить несколько пустых значений(это всё равно игрушечный, во всех смыслах, пример), чтобы начинающие саентисты покрутили датасет(лишним не будет) и подумали о методах их обработки. 

In [ ]:
data['target'].value_counts(normalize=True).plot(kind='bar')

### Зачем мы строим этот график? Нужно посмотреть как распределен целевой признак, тем более, что он не бинарный. Сразу можно подметить, что распределение не равномерное, с этим придётся, что-то делать, но пока просто примем это к сведению. 60% от всей тренировочной выборки принадлежит к классу 2, остальные признаки тоже распределены неравномерно, но их значения находятся, если можно так говорить, в пределах погрешности. 

In [ ]:
colors = ['#001c57','#50248f','#a6a6a6','#38d1ff']
sns.palplot(sns.color_palette(colors))

In [ ]:
sns.heatmap(data.corr(), cmap='YlGnBu', vmin=-1)

### Никакой корреляции между признаки не обнаружено, поэтому применять к ним какие-то статистические методы не имеет смысла.

In [ ]:
dist_feature = pd.crosstab(data['target'], data['feature_0'])
dist_feature.div(dist_feature.sum(1), axis=0).plot(kind='bar', stacked=True)

In [ ]:
dist_feature.div(dist_feature.sum(1), axis=0)

  #### График показывает распределение значений в одной фиче. "0" Используется максимальное кол-во раз, но при этом распределен равномерно во всех 4 классах.

In [ ]:
X = data.drop('target', axis = 1)
y = data.target

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=.2, random_state=88)
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

## Обозначим *Baseline* для нашей задачи и обучим первую модель без параметров.

In [ ]:
clf = CatBoostClassifier()

In [ ]:
train_pool = Pool(X_train, y_train)
valid_pool = Pool(X_valid, y_valid)

In [ ]:
clf.fit(train_pool, plot=True)

In [ ]:
plt.bar(range(len(clf.feature_importances_)), clf.feature_importances_)
plt.show()

### Видно, что некоторые фичи куда более информативнее. 

# Сделаем предикты для тренировочного датасета и запишем данные для отправки на kaggle

In [ ]:
pred = clf.predict(valid_pool)
accuracy = accuracy_score(pred, y_valid)
accuracy

In [ ]:
y_pred = clf.predict_proba(test)
y_pred

In [ ]:
submission_cat = pd.DataFrame(y_pred, columns=['Class_1', 'Class_2', 'Class_3', 'Class_4'])
submission_cat['id'] = submission['id']
submission_cat = submission_cat[['id','Class_1','Class_2','Class_3','Class_4']]
#submission_cat.to_csv('submission_cat',index=False)

#### Спасибо, за то что голосуете и приглашаю всех желающих в мой телеграмм канал: https://t.me/data_science_bleat